In [22]:
import random
import numpy as np
from Crypto.Util.number import long_to_bytes
P = 2
N = 50
E = 31337
def bytes_to_binary(s):
    bin_str = ''.join(format(b, '08b') for b in s)
    bits = [int(c) for c in bin_str]
    return bits

In [ ]:
FLAG = b'crypto{??????????????????????????}'


def generate_mat():
    while True:
        msg = bytes_to_binary(FLAG)
        msg += [random.randint(0, 1) for _ in range(N*N - len(msg))]

        rows = [msg[i::N] for i in range(N)]
        mat = Matrix(GF(2), rows)

        if mat.determinant() != 0 and mat.multiplicative_order() > 10^12:
            return mat

def load_matrix(fname):
    data = open(fname, 'r').read().strip()
    rows = [list(map(int, row)) for row in data.splitlines()]
    return Matrix(GF(P), rows)

def save_matrix(M, fname):
    open(fname, 'w').write('\n'.join(''.join(str(x) for x in row) for row in M))

mat = generate_mat()

ciphertext = mat^E # power in sage
save_matrix(ciphertext, 'flag.enc')


In [125]:
import numpy as np
from sympy import Matrix, symbols, Eq, Mod, solve

def matrix_power_gf2(matrix, exponent):
    """
    Raise a matrix to a given exponent in GF(2).
    :param matrix: The input matrix (NumPy array).
    :param exponent: The exponent to raise the matrix to.
    :return: The result of matrix raised to the exponent in GF(2).
    """
    result = np.eye(matrix.shape[0], dtype=int)  # Identity matrix
    for _ in range(exponent):
        result = np.dot(result, matrix) % 2  # Matrix multiplication mod 2
    return result

def reverse_matrix_exponentiation_gf2(exponent, result_matrix):
    """
    Reverse matrix exponentiation in GF(2).
    Finds a matrix M such that M^exponent = result_matrix in GF(2).
    
    :param exponent: The exponent.
    :param result_matrix: The result matrix in GF(2).
    :return: The matrix M that satisfies M^exponent = result_matrix.
    """
    n = result_matrix.shape[0]
    # Create symbolic matrix M with entries m_ij
    M = Matrix(n, n, lambda i, j: symbols(f'm_{i}_{j}', integer=True))
    
    # Define the matrix exponentiation equation M^exponent % 2 == result_matrix
    power_M = M**exponent
    mod_eq = Eq(Mod(power_M, 2), Matrix(result_matrix))
    
    # Solve the equation for the matrix entries (all in GF(2))
    solutions = solve(mod_eq, M)
    
    # Filter valid GF(2) solutions (all matrix entries must be 0 or 1)
    valid_solutions = []
    for sol in solutions:
        if all(v in (0, 1) for v in sol.values()):
            valid_solutions.append(sol)
    
    return valid_solutions

# Example usage:
exponent = 2
result_matrix = np.array([[1, 0], [0, 1]])  # Example result matrix in GF(2)

solutions = reverse_matrix_exponentiation_gf2(exponent, result_matrix)
print("Possible solutions for the matrix M:")
for sol in solutions:
    print(sol)

NotImplementedError: could not solve Mod(m_0_0*m_0_1 + m_0_1*m_1_1, 2)

In [62]:
from galois import GF

data = open("flag_403b981c77d39217c20390c1729b15f0.enc", 'r').read().strip()
rows = [list(map(int, row)) for row in data.splitlines()]
dat = np.array(rows, dtype=np.int8)
out = GF(P)(dat)
out

GF([[0, 0, 0, ..., 0, 0, 1],
    [1, 0, 1, ..., 0, 0, 1],
    [0, 1, 0, ..., 1, 1, 1],
    ...,
    [0, 0, 0, ..., 0, 0, 1],
    [1, 1, 1, ..., 0, 1, 1],
    [0, 0, 1, ..., 1, 1, 0]], order=2)

In [73]:
np.matmul(out, out)[1]

GF([0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
    1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
    0, 0, 0, 1], order=2)

In [123]:
size = 16
test = GF(P)(dat[0:size, 0:size])
temp = test
for i in range(10000):
    temp = np.matmul(temp, test)
    if ((temp==test).all() or (temp+test).all()): print(i)
temp

594
1189
1784
2379
2974
3569
4164
4759
5354
5949
6544
7139
7734
8329
8924
9519


GF([[0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0],
    [1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1],
    [0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1],
    [0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0],
    [1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
    [1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0],
    [1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1],
    [0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1],
    [1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0],
    [0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1],
    [0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0],
    [1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
    [0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1],
    [0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0]], order=2)

In [7]:
# ([str(i) for i in range(50)]*50)[1::N]

In [41]:
import galois
from galois import GF
import binascii
data = open("flag_403b981c77d39217c20390c1729b15f0.enc", 'r').read().strip()
rows = [list(map(int, row)) for row in data.splitlines()]
dat = np.array(rows, dtype=np.int8)
plain = dat.T.reshape((N*N))
# plain = np.array([[dat[i] for i in range(j, len(dat), 50)] for j in range(0,50)])
# flag = plain.reshape((N*N))

In [44]:
long_to_bytes(int("".join([str(plain[i]) for i in range(N*N)]), 2))

b'\x05\xc4(N*\xff(\xe7\xa6\xc8\x8a\xed\x02X\xa9\xe3\x15\xd0\xc4\xdb\xa4o\xb0\xe2cw\xbc\xfbfu\x15\xd8\xed\xb6"`\x82~M$T@\x9e4k|\xdd:\x97\xf9H\xc8\xfb\x88\x08\x17?K\x1b\xb4\x94\x86w\x9c\x1eV\x87\x89\x1a8[\xbb\x96\x87\xbe\xb3=q\xacw\x8f\xc6k~\x7f\xefvn\xe1Q\xdd>\xac\x9c\x9e\x9a\xef\xec\xca\xe30\x05\xaf*7\x8e\\\xae\xf4\xd2\xb7\xe4\xf2\x07Uz\xbf\xfc\x93>_/\xbaE\x9a\x99\x15\xfd\xe8V\x88Y\xc5\x1f\xa9%\xc6\xee\xd8\x0f\xfb\x91\x12[I1\x0bs7[\x1d\xdd%N\xdfb%d\x89G\xfd.\x17lZ\xef\x8f\xec\x8fv+\xf7]a\xaeu\xa7u\x9f\xc0X\x99\xb6W$\x18\xd6\tBg2.~\xe1c\xee\x0e\x0c\xc3\xb1c@ \x8b\n\x0e\x91\xb6j\x9eI\x98l\xe5\xf8\xa5\xbb\x15\x17\x08\xfb\xc5C\xc80\xcb\xf8\xaf9\x07Q\xf6\x1b\xb8T\xfdA\xdb4\x1f\x91j\xf5\x8f\x8f\xe2\xec\xe3\x05-d\x87\xb8\x83\xfb5\x0e\x8c\xd9)m\x1c[\xad*\xb7\xf8\x8cyU\xd0\xfe\xf1S\xaf\xd0\xd6\x1f\xed:\xe0h\x85G\xe68\xe5\xd7\xbf`w\xad\xd4\xf7\tV\xd2\x01\xcc)\x93ye\xc4j4\xaf\xabS;b(\xce'

In [58]:
import galois
from galois import GF
galois.GF(P)(np.array(rows, dtype=np.uint16))^255


GF([[255, 255, 255, ..., 255, 255, 254],
    [254, 255, 254, ..., 255, 255, 254],
    [255, 254, 255, ..., 254, 254, 254],
    ...,
    [255, 255, 255, ..., 255, 255, 254],
    [254, 254, 254, ..., 255, 254, 254],
    [255, 255, 254, ..., 254, 254, 255]], order=2)

In [19]:
"howdy! I'm flowey, flowey the"[::-2]

"etywl ywl ' ywh"